# Homework and bake-off: word-level entailment with neural networks

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Data](#Data)
  1. [Edge disjoint](#Edge-disjoint)
  1. [Word disjoint](#Word-disjoint)
1. [Baseline](#Baseline)
  1. [Representing words: vector_func](#Representing-words:-vector_func)
  1. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  1. [Classifier model](#Classifier-model)
  1. [Baseline results](#Baseline-results)
1. [Homework questions](#Homework-questions)
  1. [Hypothesis-only baseline [2 points]](#Hypothesis-only-baseline-[2-points])
  1. [Alternatives to concatenation [2 points]](#Alternatives-to-concatenation-[2-points])
  1. [A deeper network [2 points]](#A-deeper-network-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

The general problem is word-level natural language inference.

Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y = 1$ if $w_{L}$ entails $w_{R}$, otherwise $0$.

The homework questions below ask you to define baseline models for this and develop your own system for entry in the bake-off, which will take place on a held-out test-set distributed at the start of the bake-off. (Thus, all the data you have available for development is available for training your final system before the bake-off begins.)

<img src="fig/wordentail-diagram.png" width=600 alt="wordentail-diagram.png" />

## Set-up

See [the first notebook in this unit](nli_01_task_and_data.ipynb) for set-up instructions.

In [1]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import nli
import utils

In [2]:
DATA_HOME = 'data'

NLIDATA_HOME = os.path.join(DATA_HOME, 'nlidata')

wordentail_filename = os.path.join(
    NLIDATA_HOME, 'nli_wordentail_bakeoff_data.json')

GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

## Data

I've processed the data into two different train/test splits, in an effort to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample.

* `edge_disjoint`: The `train` and `dev` __edge__ sets are disjoint, but many __words__ appear in both `train` and `dev`.
* `word_disjoint`: The `train` and `dev` __vocabularies are disjoint__, and thus the edges are disjoint as well.

These are very different problems. For `word_disjoint`, there is real pressure on the model to learn abstract relationships, as opposed to memorizing properties of individual words.

In [3]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The outer keys are the  splits plus a list giving the vocabulary for the entire dataset:

In [4]:
wordentail_data.keys()

dict_keys(['edge_disjoint', 'vocab', 'word_disjoint'])

### Edge disjoint

In [5]:
wordentail_data['edge_disjoint'].keys()

dict_keys(['dev', 'train'])

This is what the split looks like; all three have this same format:

In [6]:
wordentail_data['edge_disjoint']['dev'][: 5]

[[['sweater', 'stroke'], 0],
 [['constipation', 'hypovolemia'], 0],
 [['disease', 'inflammation'], 0],
 [['herring', 'animal'], 1],
 [['cauliflower', 'outlook'], 0]]

Let's test to make sure no edges are shared between `train` and `dev`:

In [7]:
nli.get_edge_overlap_size(wordentail_data, 'edge_disjoint')

0

As we expect, a *lot* of vocabulary items are shared between `train` and `dev`:

In [8]:
nli.get_vocab_overlap_size(wordentail_data, 'edge_disjoint')

2916

This is a large percentage of the entire vocab:

In [9]:
len(wordentail_data['vocab'])

8470

Here's the distribution of labels in the `train` set. It's highly imbalanced, which will pose a challenge for learning. (I'll go ahead and reveal that the `dev` set is similarly distributed.)

In [10]:
def label_distribution(split):
    return pd.DataFrame(wordentail_data[split]['train'])[1].value_counts()

In [11]:
label_distribution('edge_disjoint')

0    14650
1     2745
Name: 1, dtype: int64

### Word disjoint

In [12]:
wordentail_data['word_disjoint'].keys()

dict_keys(['dev', 'train'])

In the `word_disjoint` split, no __words__ are shared between `train` and `dev`:

In [13]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint')

0

Because no words are shared between `train` and `dev`, no edges are either:

In [14]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint')

0

The label distribution is similar to that of `edge_disjoint`, though the overall number of examples is a bit smaller:

In [15]:
label_distribution('word_disjoint')

0    7199
1    1349
Name: 1, dtype: int64

## Baseline

Even in deep learning, __feature representation is vital and requires care!__ For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [4]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [17]:
# Any of the files in glove.6B will work here:

glove_dim = 50

glove_src = os.path.join(GLOVE_HOME, 'glove.6B.{}d.txt'.format(glove_dim))

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE = utils.glove2dict(glove_src)

def glove_vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE.get(w, randvec(w, n=glove_dim))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [8]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here; [homework question 2](#Alternatives-to-concatenation-[1-point]) below pushes you to do some exploration.

### Classifier model

For a baseline model, I chose `TorchShallowNeuralClassifier`:

In [95]:
net = TorchShallowNeuralClassifier(hidden_dim=50, max_iter=100)

### Baseline results

The following puts the above pieces together, using `vector_func=glove_vec`, since `vector_func=randvec` seems so hopelessly misguided for `word_disjoint`!

In [96]:
word_disjoint_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['word_disjoint']['train'],
    assess_data=wordentail_data['word_disjoint']['dev'], 
    model=net, 
    vector_func=glove_vec,
    vector_combo_func=vec_concatenate)

Finished epoch 100 of 100; error is 0.025118726072832942

              precision    recall  f1-score   support

           0      0.924     0.928     0.926      1910
           1      0.404     0.389     0.397       239

    accuracy                          0.868      2149
   macro avg      0.664     0.659     0.661      2149
weighted avg      0.866     0.868     0.867      2149



## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Hypothesis-only baseline [2 points]

During our discussion of SNLI and MultiNLI, we noted that a number of research teams have shown that hypothesis-only baselines for NLI tasks can be remarkably robust. This question asks you to explore briefly how this baseline effects the 'edge_disjoint' and 'word_disjoint' versions of our task.

For this problem, submit two functions:

1. A `vector_combo_func` function called `hypothesis_only` that simply throws away the premise, using the unmodified hypothesis (second) vector as its representation of the example.

1. A function called `run_hypothesis_only_evaluation` that does the following:
    1. Loops over the two conditions 'word_disjoint' and 'edge_disjoint' and the two `vector_combo_func` values `vec_concatenate` and `hypothesis_only`, calling `nli.wordentail_experiment` to train on the conditions 'train' portion and assess on its 'dev' portion, with `glove_vec` as the `vector_func`. So that the results are consistent, use an `sklearn.linear_model.LogisticRegression` with default parameters as the model.
    1. Returns a `dict` mapping `(condition_name, function_name)` pairs to the 'macro-F1' score for that pair, as returned by the call to `nli.wordentail_experiment`. (Tip: you can get the `str` name of your function `hypothesis_only` with `hypothesis_only.__name__`.)
    
The test functions `test_hypothesis_only` and `test_run_hypothesis_only_evaluation` will help ensure that your functions have the desired logic.

In [90]:
##### YOUR CODE HERE
def hypothesis_only(u, v):
    ##### YOUR CODE HERE
    return v

import sklearn

def run_hypothesis_only_evaluation():
    ##### YOUR CODE HERE
    results = {}
    conditions = ['word_disjoint', 'edge_disjoint']
    combo_funcs = [vec_concatenate, hypothesis_only]
    net = sklearn.linear_model.LogisticRegression()
    for condition in conditions:
        for combo_func in combo_funcs:
            results[(condition,combo_func.__name__)] = nli.wordentail_experiment(
                train_data=wordentail_data[condition]['train'],
                assess_data=wordentail_data[condition]['dev'], 
                model=net, 
                vector_func=glove_vec,
                vector_combo_func=combo_func)['macro-F1']
            
    return results



In [91]:
def test_hypothesis_only(hypothesis_only):
    v = hypothesis_only(1, 2)
    assert v == 2   

In [92]:
test_hypothesis_only(hypothesis_only)

In [93]:
def test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation):
    results = run_hypothesis_only_evaluation()
    assert ('word_disjoint', 'vec_concatenate') in results, \
        "The return value of `run_hypothesis_only_evaluation` does not have the intended kind of keys"
    assert isinstance(results[('word_disjoint', 'vec_concatenate')], float), \
        "The values of the `run_hypothesis_only_evaluation` result should be floats"

In [94]:
test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation)

              precision    recall  f1-score   support

           0      0.901     0.982     0.940      1910
           1      0.493     0.142     0.221       239

    accuracy                          0.888      2149
   macro avg      0.697     0.562     0.580      2149
weighted avg      0.856     0.888     0.860      2149

              precision    recall  f1-score   support

           0      0.893     0.989     0.939      1910
           1      0.382     0.054     0.095       239

    accuracy                          0.885      2149
   macro avg      0.638     0.522     0.517      2149
weighted avg      0.836     0.885     0.845      2149

              precision    recall  f1-score   support

           0      0.875     0.970     0.920      7376
           1      0.574     0.228     0.326      1321

    accuracy                          0.857      8697
   macro avg      0.725     0.599     0.623      8697
weighted avg      0.830     0.857     0.830      8697

              preci

### Alternatives to concatenation [2 points]

We've so far just used vector concatenation to represent the premise and hypothesis words. This question asks you to explore two simple alternative:

1. Write a function `vec_diff` that, for a given pair of vector inputs `u` and `v`, returns the element-wise difference between `u` and `v`.

1. Write a function `vec_max` that, for a given pair of vector inputs `u` and `v`, returns the element-wise max values between `u` and `v`.

You needn't include your uses of `nli.wordentail_experiment` with these functions, but we assume you'll be curious to see how they do!

In [54]:
def vec_diff(u, v):
    ##### YOUR CODE HERE
    return u-v



    
def vec_max(u, v):
    ##### YOUR CODE HERE
    return np.max([u,v],axis=0)



In [55]:
def test_vec_diff(vec_diff):
    u = np.array([10.2, 8.1])
    v = np.array([1.2, -7.1])
    result = vec_diff(u, v)
    expected = np.array([9.0, 15.2])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [56]:
test_vec_diff(vec_diff)

In [57]:
def test_vec_max(vec_max):
    u = np.array([1.2,  8.1])
    v = np.array([10.2, -7.1])
    result = vec_max(u, v)
    expected = np.array([10.2, 8.1])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [58]:
test_vec_max(vec_max)

### A deeper network [2 points]

It is very easy to subclass `TorchShallowNeuralClassifier` if all you want to do is change the network graph: all you have to do is write a new `define_graph`. If your graph has new arguments that the user might want to set, then you should also redefine `__init__` so that these values are accepted and set as attributes.

For this question, please subclass `TorchShallowNeuralClassifier` so that it defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
r_{1} &= \textbf{Bernoulli}(1 - \textbf{dropout\_prob}, n) \\
d_{1} &= r_1 * h_{1} \\
h_{2} &= f(d_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

Here, $r_{1}$ and $d_{1}$ define a dropout layer: $r_{1}$ is a random binary vector of dimension $n$, where the probability of a value being $1$ is given by $1 - \textbf{dropout_prob}$. $r_{1}$ is multiplied element-wise by our first hidden representation, thereby zeroing out some of the values. The result is fed to the user's activation function $f$, and the result of that is fed through another linear layer to produce $h_{3}$. (Inside `TorchShallowNeuralClassifier`, $h_{3}$ is the basis for a softmax classifier, so no activation function is applied to it.)

For your implementation, please use `nn.Sequential`, `nn.Linear`, and `nn.Dropout` to define the required layers.

For comparison, using this notation, `TorchShallowNeuralClassifier` defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
h_{2} &= f(h_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

The following code starts this sub-class for you, so that you can concentrate on `define_graph`. Be sure to make use of `self.dropout_prob`

For this problem, submit just your completed  `TorchDeepNeuralClassifier`. You needn't evaluate it, though we assume you will be keen to do that!

You can use `test_TorchDeepNeuralClassifier` to ensure that your network has the intended structure.

In [101]:
import torch.nn as nn

class TorchDeepNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)
    
    def define_graph(self):
        """Complete this method!
        
        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you 
        write yourself, as in `torch_rnn_classifier`, or the outpiut of 
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.
        
        """
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))



    

##### YOUR CODE HERE    
net = TorchDeepNeuralClassifier(hidden_dim=50, max_iter=100)
word_disjoint_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['word_disjoint']['train'],
    assess_data=wordentail_data['word_disjoint']['dev'], 
    model=net, 
    vector_func=glove_vec,
    vector_combo_func=vec_concatenate)

Finished epoch 100 of 100; error is 2.5403620302677155

              precision    recall  f1-score   support

           0      0.902     0.990     0.944      1910
           1      0.623     0.138     0.226       239

    accuracy                          0.895      2149
   macro avg      0.762     0.564     0.585      2149
weighted avg      0.871     0.895     0.864      2149



In [66]:
def test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier):
    dropout_prob = 0.55
    assert hasattr(TorchDeepNeuralClassifier(), "dropout_prob"), \
        "TorchDeepNeuralClassifier must have an attribute `dropout_prob`."
    try:
        inst = TorchDeepNeuralClassifier(dropout_prob=dropout_prob)
    except TypeError:
        raise TypeError("TorchDeepNeuralClassifier must allow the user "
                        "to set `dropout_prob` on initialization")
    inst.input_dim = 10
    inst.n_classes_ = 5
    graph = inst.define_graph()
    assert len(graph) == 4, \
        "The graph should have 4 layers; yours has {}".format(len(graph))    
    expected = {
        0: 'Linear',
        1: 'Dropout',
        2: 'Tanh',
        3: 'Linear'}
    for i, label in expected.items():
        name = graph[i].__class__.__name__
        assert label in name, \
            "The {} layer of the graph should be a {} layer; yours is {}".format(i, label, name)
    assert graph[1].p == dropout_prob, \
        "The user's value for `dropout_prob` should be the value of `p` for the Dropout layer."

In [67]:
test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier)

### Your original system [3 points]

This is a simple dataset, but our focus on the 'word_disjoint' condition ensures that it's a challenging one, and there are lots of modeling strategies one might adopt. 

You are free to do whatever you like. We require only that your system differ in some way from those defined in the preceding questions. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

Keep in mind that, for the bake-off evaluation, the 'edge_disjoint' portions of the data are off limits. You can, though, train on the combination of the 'word_disjoint' 'train' and 'dev' portions. You are free to use different pretrained word vectors and the like. Please do not introduce additional entailment datasets into your training data, though.

Please embed your code in this notebook so that we can rerun it.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# Enter your system description in this cell.
# For the model, we implemented a deeper fully connected network with 2 hidden layers. Then we do a grid 
# search on all possible hyper-parameters including globe vector dimension (50, 100, 200, 300), dropout ratio 
# (0.3, 0.5, 0.7), network channel number (50, 100, 200), and number of iterations for training (100, 200, 300, 500).
# It turns out the best combination is using glove dimension 100, dropout ratio 0.5, network channel number 200, 
# and training iteration 200. The resulted F1 macro average number is 0.7068. We've also tried using retrofit vector
# and Alberta word embedding, none of them gives better results. 


# My peak score was: 0.7068
# This is my code
import torch.nn as nn

class TorchCustomNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)
    
    def define_graph(self):
        """Complete this method!
        
        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you 
        write yourself, as in `torch_rnn_classifier`, or the outpiut of 
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.
        
        """
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))


if 'IS_GRADESCOPE_ENV' not in os.environ:
    glove_dim = 100
    dropout = 0.5
    dim = 200
    iters = 200
    
    def glove_vec(w):    
        """Return `w`'s GloVe representation if available, else return 
        a random vector."""
        return GLOVE.get(w, randvec(w, n=glove_dim))
    glove_src = os.path.join(GLOVE_HOME, 'glove.6B.{}d.txt'.format(glove_dim))

    # Creates a dict mapping strings (words) to GloVe vectors:
    GLOVE = utils.glove2dict(glove_src)


    net = TorchCustomNeuralClassifier(hidden_dim=dim, max_iter=iters, dropout_prob=dropout)
    word_disjoint_experiment = nli.wordentail_experiment(
        train_data=wordentail_data['word_disjoint']['train'],
        assess_data=wordentail_data['word_disjoint']['dev'], 
        model=net, 
        vector_func=glove_vec,
        vector_combo_func=vec_concatenate)

# Please do not remove this comment.

In [9]:

# from nltk.corpus import wordnet as wn
# import retrofitting
# from retrofitting import Retrofitter

# glove_dim = 300

# glove_src = os.path.join(GLOVE_HOME, 'glove.6B.{}d.txt'.format(glove_dim))

# # Creates a dict mapping strings (words) to GloVe vectors:
# GLOVE = utils.glove2dict(glove_src)

# X_glove = pd.DataFrame(GLOVE).T

# def get_wordnet_edges():
#     edges = defaultdict(set)
#     for ss in wn.all_synsets():
#         lem_names = {lem.name() for lem in ss.lemmas()}
#         for lem in lem_names:
#             edges[lem] |= lem_names
#     return edges

# wn_edges = get_wordnet_edges()


# def convert_edges_to_indices(edges, Q):
#     lookup = dict(zip(Q.index, range(Q.shape[0])))
#     index_edges = defaultdict(set)
#     for start, finish_nodes in edges.items():
#         s = lookup.get(start)
#         if s:
#             f = {lookup[n] for n in finish_nodes if n in lookup}
#             if f:
#                 index_edges[s] = f
#     return index_edges
# wn_index_edges = convert_edges_to_indices(wn_edges, X_glove)

# wn_retro = Retrofitter(verbose=True)
# X_retro = wn_retro.fit(X_glove, wn_index_edges)

# def glove_vec(w):    
#     """Return `w`'s GloVe representation if available, else return 
#     a random vector."""
#     return GLOVE.get(w, randvec(w, n=glove_dim))

# net = TorchShallowNeuralClassifier(hidden_dim=50, max_iter=200)
# word_disjoint_experiment = nli.wordentail_experiment(
#     train_data=wordentail_data['word_disjoint']['train'],
#     assess_data=wordentail_data['word_disjoint']['dev'], 
#     model=net, 
#     vector_func=glove_vec,
#     vector_combo_func=vec_concatenate)

# import torch
# roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
# # roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
# roberta.eval()

# # tokens = roberta.encode(['hello', 'hi'])
# # print(tokens)
# # last_layer_features = roberta.extract_features(tokens)
# # print(last_layer_features.shape)

# bert_mapping = {}
# def bert_vec(w):    
#     """Return `w`'s GloVe representation if available, else return 
#     a random vector."""
#     return bert_mapping[w]

# def bert_vec_mapping():
#     for w in wordentail_data['vocab']:
#         tokens = roberta.encode(w)
#         last_layer_features = roberta.extract_features(tokens)
#         bert_mapping[w] = last_layer_features[0,1,:].detach().numpy().shape

# bert_vec_mapping()


# glove_dim = 100


# # net = TorchShallowNeuralClassifier(hidden_dim=300, max_iter=1000)
# max_f1 = 0.0
# max_param = None
# for glove_dim in [50, 100, 200, 300]:              
#     def glove_vec(w):    
#         """Return `w`'s GloVe representation if available, else return 
#         a random vector."""
#         return GLOVE.get(w, randvec(w, n=glove_dim))
#     glove_src = os.path.join(GLOVE_HOME, 'glove.6B.{}d.txt'.format(glove_dim))

#     # Creates a dict mapping strings (words) to GloVe vectors:
#     GLOVE = utils.glove2dict(glove_src)
#     for dropout in [0.3,0.5,0.7]:
#         for dim in [50, 100, 200]:
#             for iters in [100, 200, 300, 500]:  


#                 net = TorchCustomNeuralClassifier(hidden_dim=dim, max_iter=iters, dropout_prob=dropout)
#                 word_disjoint_experiment = nli.wordentail_experiment(
#                     train_data=wordentail_data['word_disjoint']['train'],
#                     assess_data=wordentail_data['word_disjoint']['dev'], 
#                     model=net, 
#                     vector_func=glove_vec,
#                     vector_combo_func=vec_concatenate)
#                 print(dropout, dim, iters, glove_dim, word_disjoint_experiment['macro-F1'])
#                 if word_disjoint_experiment['macro-F1']>max_f1:
#                     max_f1 = word_disjoint_experiment['macro-F1']
#                     max_param = (dropout, dim, iters, glove_dim)

# print("----")
# print(max_param)
# print(max_f1)

Finished epoch 100 of 100; error is 1.1998164653778076

              precision    recall  f1-score   support

           0      0.917     0.968     0.941      1910
           1      0.534     0.297     0.382       239

    accuracy                          0.893      2149
   macro avg      0.725     0.632     0.662      2149
weighted avg      0.874     0.893     0.879      2149

0.3 50 100 50 0.6615683149009361


Finished epoch 200 of 200; error is 0.9480023384094238

              precision    recall  f1-score   support

           0      0.923     0.968     0.945      1910
           1      0.575     0.351     0.436       239

    accuracy                          0.899      2149
   macro avg      0.749     0.660     0.690      2149
weighted avg      0.884     0.899     0.888      2149

0.3 50 200 50 0.6904537323141974


Finished epoch 300 of 300; error is 0.8455384373664856

...

              precision    recall  f1-score   support

           0      0.911     0.971     0.940      1910
           1      0.513     0.243     0.330       239

    accuracy                          0.890      2149
   macro avg      0.712     0.607     0.635      2149
weighted avg      0.867     0.890     0.872      2149

0.7 200 200 300 0.6348690273234114


Finished epoch 300 of 300; error is 0.9246631935238838

              precision    recall  f1-score   support

           0      0.910     0.977     0.942      1910
           1      0.557     0.226     0.321       239

    accuracy                          0.894      2149
   macro avg      0.733     0.602     0.632      2149
weighted avg      0.871     0.894     0.873      2149

0.7 200 300 300 0.6319409389197375


Finished epoch 500 of 500; error is 0.8570183813571933

              precision    recall  f1-score   support

           0      0.907     0.984     0.944      1910
           1      0.603     0.197     0.297       239

    accuracy                          0.896      2149
   macro avg      0.755     0.590     0.620      2149
weighted avg      0.873     0.896     0.872      2149

0.7 200 500 300 0.6202569460457679
----
(0.5, 200, 200, 100)
0.7068196235259503


## Bake-off [1 point]

The goal of the bake-off is to achieve the highest macro-average F1 score on __word_disjoint__, on a test set that we will make available at the start of the bake-off. The announcement will go out on the discussion forum. To enter, you'll be asked to run `nli.bake_off_evaluation` on the output of your chosen `nli.wordentail_experiment` run. 

The cells below this one constitute your bake-off entry.

The rules described in the [Your original system](#Your-original-system-[3-points]) homework question are also in effect for the bake-off.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code into this cell. 
# Please do not remove this comment.
##### YOUR CODE HERE




In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-avg f1 value as reported by the code above. 
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.

##### YOUR CODE HERE


